In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210124.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,1/19/2021 20:36:40,The General (1926),The Train (1964),The Taking of Pelham One Two Three (1974),Strangers On A Train (1951),Murder on the Orient Express (1974),Train to Busan (2016),THE WOLVES ARE AT OL' GIL'S DOOR,THIS CARTOON AINT FOR CHILDREN,[Default to Nicolas Cage],PAUL GIAMATTI,SUGGESTED ON A PODCAST EPISODE,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC."
1,1/19/2021 20:56:16,The Taking of Pelham One Two Three (1974),Strangers On A Train (1951),The General (1926),The Train (1964),Train to Busan (2016),Murder on the Orient Express (1974),"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.",THE WOLVES ARE AT OL' GIL'S DOOR,THIS CARTOON AINT FOR CHILDREN,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,SUGGESTED ON A PODCAST EPISODE,PAUL GIAMATTI,[Default to Nicolas Cage]
2,1/19/2021 20:56:36,The Train (1964),The Taking of Pelham One Two Three (1974),Murder on the Orient Express (1974),NaN,NaN,NaN,THE WOLVES ARE AT OL' GIL'S DOOR,"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.",SUGGESTED ON A PODCAST EPISODE,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,THIS CARTOON AINT FOR CHILDREN,[Default to Nicolas Cage],PAUL GIAMATTI
3,1/19/2021 20:57:16,The Taking of Pelham One Two Three (1974),Strangers On A Train (1951),Train to Busan (2016),The Train (1964),Murder on the Orient Express (1974),The General (1926),SUGGESTED ON A PODCAST EPISODE,THIS CARTOON AINT FOR CHILDREN,"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.",THE WOLVES ARE AT OL' GIL'S DOOR,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,PAUL GIAMATTI,[Default to Nicolas Cage]
4,1/19/2021 21:20:44,Train to Busan (2016),Murder on the Orient Express (1974),Strangers On A Train (1951),NaN,NaN,NaN,PAUL GIAMATTI,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,[Default to Nicolas Cage],THIS CARTOON AINT FOR CHILDREN,NaN,NaN,NaN
5,1/20/2021 9:19:50,Murder on the Orient Express (1974),Train to Busan (2016),The Taking of Pelham One Two Three (1974),Strangers On A Train (1951),The General (1926),The Train (1964),THIS CARTOON AINT FOR CHILDREN,PAUL GIAMATTI,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,[Default to Nicolas Cage],THE WOLVES ARE AT OL' GIL'S DOOR,"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.",SUGGESTED ON A PODCAST EPISODE
6,1/20/2021 18:09:19,Murder on the Orient Express (1974),Train to Busan (2016),The Taking of Pelham One Two Three (1974),Strangers On A Train (1951),The General (1926),NaN,THE WOLVES ARE AT OL' GIL'S DOOR,"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.",THIS CARTOON AINT FOR CHILDREN,PAUL GIAMATTI,[Default to Nicolas Cage],NaN,NaN
7,1/21/2021 8:54:50,Murder on the Orient Express (1974),The Taking of Pelham One Two Three (1974),The Train (1964),The General (1926),Strangers On A Train (1951),Train to Busan (2016),[Default to Nicolas Cage],THE WOLVES ARE AT OL' GIL'S DOOR,THIS CARTOON AINT FOR CHILDREN,IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME,"GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.",PAUL GIAMATTI,SUGGESTED ON A PODCAST EPISODE
8,1/21/2021 9:05:49,The General (1926),The Taking of Pelham One Two Three (1974),Murder on the Orient Express 

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['The General (1926)', 'The Train (1964)',
       'The Taking of Pelham One Two Three (1974)',
       'Strangers On A Train (1951)',
       'Murder on the Orient Express (1974)', 'Train to Busan (2016)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'The General (1926)': <Candidate('The General (1926)')>,
 'The Train (1964)': <Candidate('The Train (1964)')>,
 'The Taking of Pelham One Two Three (1974)': <Candidate('The Taking of Pelham One Two Three (1974)')>,
 'Strangers On A Train (1951)': <Candidate('Strangers On A Train (1951)')>,
 'Murder on the Orient Express (1974)': <Candidate('Murder on the Orient Express (1974)')>,
 'Train to Busan (2016)': <Candidate('Train to Busan (2016)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The General (1926), The Train (1964), The Taking of Pelham One Two Three (1974), Strangers On A Train (1951), Murder on the Orient Express (1974), Train to Busan (2016))>,
 <Ballot(The Taking of Pelham One Two Three (1974), Strangers On A Train (1951), The General (1926), The Train (1964), Train to Busan (2016), Murder on the Orient Express (1974))>,
 <Ballot(The Train (1964), The Taking of Pelham One Two Three (1974), Murder on the Orient Express (1974))>,
 <Ballot(The Taking of Pelham One Two Three (1974), Strangers On A Train (1951), Train to Busan (2016), The Train (1964), Murder on the Orient Express (1974), The General (1926))>,
 <Ballot(Train to Busan (2016), Murder on the Orient Express (1974), Strangers On A Train (1951))>,
 <Ballot(Murder on the Orient Express (1974), Train to Busan (2016), The Taking of Pelham One Two Three (1974), Strangers On A Train (1951), The General (1926), The Train (1964))>,
 <Ballot(Murder on the Orient Express (1974), Train to Busan (2016)

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
Murder on the Orient Express (1974)              3  Hopeful
The Taking of Pelham One Two Three (1974)        2  Hopeful
The General (1926)                               2  Hopeful
Train to Busan (2016)                            1  Rejected
The Train (1964)                                 1  Rejected
Strangers On A Train (1951)                      0  Rejected

ROUND 2
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
Murder on the Orient Express (1974)              4  Hopeful
The Taking of Pelham One Two Three (1974)        3  Hopeful
The General (1926)                               2  Rejected
Train to Busan (2016)                            0  Rejected
The Train (1964)                                 0  Rejected
Strangers On A Train (1951)                      0  Rejected

FINAL RESULT
Ca

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                           Votes  Status
------------------------------------------------  -------  --------
THE WOLVES ARE AT OL' GIL'S DOOR                        3  Hopeful
PAUL GIAMATTI                                           2  Hopeful
THIS CARTOON AINT FOR CHILDREN                          1  Hopeful
GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, INC.           1  Hopeful
[Default to Nicolas Cage]                               1  Hopeful
SUGGESTED ON A PODCAST EPISODE                          1  Rejected
IF YOU AIN’T GOT NO MONEY TAKE YO BROKE ASS HOME        0  Rejected

ROUND 2
Candidate                                           Votes  Status
------------------------------------------------  -------  --------
THE WOLVES ARE AT OL' GIL'S DOOR                        3  Hopeful
THIS CARTOON AINT FOR CHILDREN                          2  Hopeful
PAUL GIAMATTI                                           2  Hopeful
GOLAN-GLOBUS A GO-GO!: THE CANNON GROUP, IN